<a href="https://colab.research.google.com/github/zoraizmohammad/FlavorFlow/blob/main/data/dataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Processing**

**Data Insertion**

View Tasks here: https://github.com/zoraizmohammad/FlavorFlow/blob/main/weeklyTasks/week2.md#to-complete-step-1-data-collection-and-preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.4 MB/s eta 0:00:00


In [3]:
import pandas as pd

def view(file_path, num_columns):
  df = pd.read_csv(file_path)
  print(df.iloc[:, :num_columns])

In [4]:
raw_annotation = pd.read_csv('/content/drive/MyDrive/FlavorSync/Data/full_dataset.csv')
# Based on who's using copy and paste in path. Refrence:
# RP MZ: /content/drive/MyDrive/FlavorSync/Data/full_dataset.csv
# RP GM:
# RP RM:

In [ ]:
raw_annotation

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


## **Data Cleaning & Export**

In [10]:
import pandas as pd
import nltk
from nltk.corpus import words, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from quantulum3 import parser
from fuzzywuzzy import fuzz, process

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/FlavorSync/Data/full_dataset.csv')

#Remove Duplicates
# Filter by source and remove duplicate recipes
data = data[data['source'] == "Gathered"].drop_duplicates()

# Normalize Text
# Download necessary NLTK data
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Create a lemmatizer instance
lemmatizer = WordNetLemmatizer()

# Base ingredient names
base_ingredients = [
    "bell pepper", "chili", "cilantro", "eggplant", "zucchini", "potato", "tomato",
    "onion", "garlic", "carrot", "mushroom", "cabbage", "lettuce", "spinach"
]

# Ingredient mapping dictionary
ingredient_mapping = {ingredient: ingredient for ingredient in base_ingredients}

# Function to automate ingredient mapping using fuzzy matching
def create_ingredient_mapping(ingredient_list, base_ingredients, threshold=80):
    for ingredient in ingredient_list:
        # Find the best match from the base ingredients using fuzzy matching
        match, score = process.extractOne(ingredient, base_ingredients)
        if score >= threshold:  # If the similarity score is above the threshold
            ingredient_mapping[ingredient] = match

# Get a list of unique ingredients from the dataset
unique_ingredients = set()
data['ingredients'].apply(lambda x: unique_ingredients.update(word_tokenize(x.lower())))

# Generate ingredient mappings
create_ingredient_mapping(unique_ingredients, base_ingredients)

# Function to normalize ingredients
def normalize_ingredients(text):
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase the text
    normalized_tokens = []

    for token in tokens:
        # Lemmatize each token and map using the ingredient dictionary
        lemmatized_word = lemmatizer.lemmatize(token)
        if lemmatized_word in ingredient_mapping:
            normalized_tokens.append(ingredient_mapping[lemmatized_word])
        else:
            normalized_tokens.append(lemmatized_word)

    return ' '.join(normalized_tokens)

# Apply normalization to the 'ingredients' column
data['ingredients'] = data['ingredients'].apply(normalize_ingredients)

#Spell-Check and Correct Typos
english_words = set(words.words())

def correct_spelling(text):
    tokens = word_tokenize(text)
    corrected_tokens = [token if token in english_words else token for token in tokens]
    return ' '.join(corrected_tokens)

# Apply spell-check to the 'steps' column
data['steps'] = data['steps'].apply(correct_spelling)

# Standardize Units
def standardize_units(text):
    quantities = parser.parse(text)
    standardized_text = " ".join([str(quantity) for quantity in quantities])
    return standardized_text

# Apply unit standardization
data['ingredients'] = data['ingredients'].apply(standardize_units)

# Save the cleaned data
data.to_csv("/content/drive/MyDrive/FlavorSync/Data/cleaned_data.csv", index=False)

data.head()


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


# **Tokenize Ingredients & Steps**